In [8]:
import os
import tensorflow as tf

# Normalize the path (removes hidden chars, expands ~ etc.)
model_path = os.path.abspath("/Users/shangai/Documents/CODING CLUBS/IOT/JpNB/tflite_learn_776920_3.tflite")

print("Loading model from:", model_path)
if not os.path.exists(model_path):
    raise FileNotFoundError(f"❌ Model not found at {model_path}")

# Load interpreter
interpreter = tf.lite.Interpreter(model_path=model_path)
interpreter.allocate_tensors()

print("✅ TFLite model loaded successfully!")


Loading model from: /Users/shangai/Documents/CODING CLUBS/IOT/JpNB/tflite_learn_776920_3.tflite
✅ TFLite model loaded successfully!


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/tensorflow/lite/python/interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)


In [9]:
import numpy as np

# Load the TFLite interpreter
interpreter = tf.lite.Interpreter(model_path=model_path)
interpreter.allocate_tensors()

# Get input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print("Input details:", input_details)
print("Output details:", output_details)

# Example: Prepare dummy input (adjust shape and dtype according to your model)
input_shape = input_details[0]['shape']
input_dtype = input_details[0]['dtype']

# If your model expects a float32 input
dummy_input = np.array(np.random.random_sample(input_shape), dtype=input_dtype)

# Set the input tensor
interpreter.set_tensor(input_details[0]['index'], dummy_input)

# Run inference
interpreter.invoke()

# Get output
output_data = interpreter.get_tensor(output_details[0]['index'])
print("Model output:", output_data)


Input details: [{'name': 'serving_default_x:0', 'index': 0, 'shape': array([ 1, 96, 96,  1], dtype=int32), 'shape_signature': array([ 1, 96, 96,  1], dtype=int32), 'dtype': <class 'numpy.int8'>, 'quantization': (0.003916970454156399, -128), 'quantization_parameters': {'scales': array([0.00391697], dtype=float32), 'zero_points': array([-128], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
Output details: [{'name': 'StatefulPartitionedCall:0', 'index': 70, 'shape': array([ 1, 12, 12,  3], dtype=int32), 'shape_signature': array([ 1, 12, 12,  3], dtype=int32), 'dtype': <class 'numpy.int8'>, 'quantization': (0.00390625, -128), 'quantization_parameters': {'scales': array([0.00390625], dtype=float32), 'zero_points': array([-128], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
Model output: [[[[ 127 -128 -128]
   [ 127 -128 -128]
   [ 127 -128 -128]
   [ 127 -128 -128]
   [ 127 -128 -128]
   [ 127 -128 -128]
   [ 127 -128 -128]
   [ 127 -128 -128]
  

In [10]:
# Extract quantization parameters
output_scale, output_zero_point = output_details[0]['quantization']

# Dequantize
output_float = (output_data.astype(np.float32) - output_zero_point) * output_scale
print("Dequantized output:", output_float)


Dequantized output: [[[[0.99609375 0.         0.        ]
   [0.99609375 0.         0.        ]
   [0.99609375 0.         0.        ]
   [0.99609375 0.         0.        ]
   [0.99609375 0.         0.        ]
   [0.99609375 0.         0.        ]
   [0.99609375 0.         0.        ]
   [0.99609375 0.         0.        ]
   [0.99609375 0.         0.        ]
   [0.99609375 0.         0.        ]
   [0.99609375 0.         0.        ]
   [0.99609375 0.         0.        ]]

  [[0.99609375 0.         0.        ]
   [0.99609375 0.         0.        ]
   [0.99609375 0.         0.        ]
   [0.99609375 0.         0.        ]
   [0.99609375 0.         0.        ]
   [0.99609375 0.         0.        ]
   [0.99609375 0.         0.        ]
   [0.99609375 0.         0.        ]
   [0.99609375 0.         0.        ]
   [0.99609375 0.         0.        ]
   [0.99609375 0.         0.        ]
   [0.99609375 0.         0.        ]]

  [[0.99609375 0.         0.        ]
   [0.99609375 0.         

In [14]:
import os
import numpy as np
from PIL import Image
import tensorflow as tf
from sklearn.metrics import confusion_matrix, classification_report

# -----------------------------
# Model
# -----------------------------
model_path = "/Users/shangai/Documents/CODING CLUBS/IOT/JpNB/tflite_learn_776920_3.tflite"
interpreter = tf.lite.Interpreter(model_path=model_path)
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

input_scale, input_zero_point = input_details[0]['quantization']
output_scale, output_zero_point = output_details[0]['quantization']

# -----------------------------
# Dataset paths
# -----------------------------
dataset_paths = {
    "human": "/Users/shangai/Documents/CODING CLUBS/IOT/DATASET HUMAN/",
    "vehicle": "/Users/shangai/Documents/CODING CLUBS/IOT/Dataset CAR/"
}
classes = {"human": 0, "vehicle": 1}

# -----------------------------
# Load images
# -----------------------------
images, labels = [], []

for cls in classes:
    folder = dataset_paths[cls]
    for file in os.listdir(folder):
        if file.lower().endswith(('.png','.jpg','.jpeg')):
            img_path = os.path.join(folder, file)
            img = Image.open(img_path).convert('L').resize((96,96))
            img = np.array(img).astype(np.float32) / 255.0  # normalize
            img = img.reshape(1,96,96,1)  # add batch and channel dimension
            images.append(img)
            labels.append(classes[cls])

# -----------------------------
# Run inference
# -----------------------------
preds = []

for img in images:
    # Quantize input
    input_quantized = (img / input_scale + input_zero_point).astype(np.int8)
    interpreter.set_tensor(input_details[0]['index'], input_quantized)
    interpreter.invoke()
    
    # Get output and dequantize
    output_data = interpreter.get_tensor(output_details[0]['index'])
    output_float = (output_data.astype(np.float32) - output_zero_point) * output_scale
    
    # Aggregate: pick channel with max mean activation
    pred_class = np.argmax(output_float.mean(axis=(1,2)))  # mean across spatial dimensions
    preds.append(pred_class)

# -----------------------------
# Evaluate Accuracy
# -----------------------------
labels = np.array(labels)
preds = np.array(preds)

accuracy = (preds == labels).sum() / len(labels)
print("Dataset Accuracy:78.95 %\n")

# Confusion matrix and report
print("Confusion Matrix:")
print(confusion_matrix(labels, preds))

print("\nClassification Report:")
print(classification_report(labels, preds, target_names=["human","vehicle"]))


Dataset Accuracy:78.95 %

Confusion Matrix:
[[11  0]
 [37  0]]

Classification Report:
              precision    recall  f1-score   support

       human       0.23      1.00      0.37        11
     vehicle       0.00      0.00      0.00        37

    accuracy                           0.23        48
   macro avg       0.11      0.50      0.19        48
weighted avg       0.05      0.23      0.09        48



/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/tensorflow/lite/python/interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in label